# Proprocessing of Bedmap1, Bedmap2, and Bedmap3 data

This notebook contains all preprocessing steps applied to the raw Bedmap data for full **reproducibility**.

We download all data directly from the [UK Polar Data Centre](https://www.bas.ac.uk/data/uk-pdc/). The links to the *standardised csv* files of Bedmap1, Bedmap2 and Bedmap3 respectively are listed [on this Bristish Antarctic Survey (BAS) webpage](https://www.bas.ac.uk/project/bedmap/#data). We use the wget shell command and the urls to the data sets to download the files programmatically. Be aware of the size of the data sets. The collection of standarised csv files that make up Bedmap3 are 1.25 GB for instance. In the following we download all csvs, combine all data into one, perform some commonsense quality checks and cleaning, and crop the data to our area of interest. You only need to define a target directly to house the data and run the notebook below. If you have trouble with this approach, also see this [BAS resource](https://antarctica.github.io/PDC_GeophysicsBook/BEDMAP/Downloading_the_Bedmap_data.html) for useful information. Furthermore, this [Github repository](https://github.com/kimbente/bedmap) contains some additional analysis.


## Data structure

We create the following folder structure to download the files into the respective locations:

.  
└── bedmap_raw_data/  
    ├── bedmap1_raw_data/  
    │   └── BEDMAP1_1966-2000_AIR_BM1.csv  
    ├── bedmap2_raw_data  
    └── bedmap3_raw_data  


# DEFINE YOUR PATH to data directory

Define the path to a new, empty directory to download all the data into. You can optionally use the mkdir command below to generate this directory.

In [ ]:
# CHANGE THIS AS NEEDED
# WARNING: This script will download a large amounts of data
path_to_bedmap_data_folder = "/home/kim/data/bedmap_raw_data"

# Make this directory locally if you haven't already
!mkdir -p "{path_to_bedmap_data_folder}"

## Create subdirectories 

In [2]:
path_to_bedmap1_data_folder = f"{path_to_bedmap_data_folder}/bedmap1_raw_data"
path_to_bedmap2_data_folder = f"{path_to_bedmap_data_folder}/bedmap2_raw_data"
path_to_bedmap3_data_folder = f"{path_to_bedmap_data_folder}/bedmap3_raw_data"

!mkdir -p "{path_to_bedmap1_data_folder}"
!mkdir -p "{path_to_bedmap2_data_folder}"
!mkdir -p "{path_to_bedmap3_data_folder}"

## Download Bedmap1

Bedmap1 contains only one .csv file.

In [4]:
path_to_bedmap1_csv = f"{path_to_bedmap1_data_folder}/BEDMAP1_1966-2000_AIR_BM1.csv"
bedmap1_url = "https://ramadda.data.bas.ac.uk/repository/entry/get/BEDMAP1_1966-2000_AIR_BM1.csv?entryid=synth%3Af64815ec-4077-4432-9f55-0ce230f46029%3AL0JFRE1BUDFfMTk2Ni0yMDAwX0FJUl9CTTEuY3N2"

# Download the 156.24MB CSV file from the URL
!wget -O "{path_to_bedmap1_csv}" "{bedmap1_url}"

--2025-05-15 09:40:25--  https://ramadda.data.bas.ac.uk/repository/entry/get/BEDMAP1_1966-2000_AIR_BM1.csv?entryid=synth%3Af64815ec-4077-4432-9f55-0ce230f46029%3AL0JFRE1BUDFfMTk2Ni0yMDAwX0FJUl9CTTEuY3N2
Resolving ramadda.data.bas.ac.uk (ramadda.data.bas.ac.uk)... 194.66.5.50
Connecting to ramadda.data.bas.ac.uk (ramadda.data.bas.ac.uk)|194.66.5.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 163826882 (156M) [text/csv]
Saving to: '/home/kim/data/bedmap_raw_data/bedmap1_raw_data/BEDMAP1_1966-2000_AIR_BM1.csv’

/home/kim/data/bedm 100%[===================>] 156.24M  8.95MB/s    in 18s     

2025-05-15 09:40:44 (8.89 MB/s) - '/home/kim/data/bedmap_raw_data/bedmap1_raw_data/BEDMAP1_1966-2000_AIR_BM1.csv’ saved [163826882/163826882]



## Download Bedmap2

Bedmap2 contains many csv files.

In [6]:
# 542.16M
# provide name for download
!wget -P "{path_to_bedmap2_data_folder}/bedmap2.zip" "https://ramadda.data.bas.ac.uk/repository/entry/show?entryid=2fd95199-365e-4da1-ae26-3b6d48b3e6ac&output=zip.tree"

# unzip into the directory it is in
!unzip "{path_to_bedmap2_data_folder}/bedmap2.zip" -d "{path_to_bedmap2_data_folder}"

--2025-05-15 09:52:44--  https://ramadda.data.bas.ac.uk/repository/entry/show?entryid=2fd95199-365e-4da1-ae26-3b6d48b3e6ac&output=zip.tree
Resolving ramadda.data.bas.ac.uk (ramadda.data.bas.ac.uk)... 194.66.5.50
Connecting to ramadda.data.bas.ac.uk (ramadda.data.bas.ac.uk)|194.66.5.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [multipart/x-zip]
Saving to: '/home/kim/data/bedmap_raw_data/bedmap2_raw_data/show?entryid=2fd95199-365e-4da1-ae26-3b6d48b3e6ac&output=zip.tree’

show?entryid=2fd951     [            <=>     ] 542.16M  6.16MB/s    in 97s     

2025-05-15 09:54:24 (5.60 MB/s) - '/home/kim/data/bedmap_raw_data/bedmap2_raw_data/show?entryid=2fd95199-365e-4da1-ae26-3b6d48b3e6ac&output=zip.tree’ saved [568496192]



In [12]:
# unzip into the directory it is in
!unzip "{path_to_bedmap2_data_folder}/bedmap2.zip" -d "{path_to_bedmap2_data_folder}"

# Move all files from the nested folder to the main bedmap2 folder
!mv "{path_to_bedmap2_data_folder}/BEDMAP2 - Ice thickness, bed and surface elevation for Antarctica - standardised data points/"* "{path_to_bedmap2_data_folder}/"

# Remove the now-empty folder
!rmdir "{path_to_bedmap2_data_folder}/BEDMAP2 - Ice thickness, bed and surface elevation for Antarctica - standardised data points"

Archive:  /home/kim/data/bedmap_raw_data/bedmap2_raw_data/bedmap2.zip
  inflating: /home/kim/data/bedmap_raw_data/bedmap2_raw_data/BEDMAP2 - Ice thickness, bed and surface elevation for Antarctica - standardised data points/AWI_1994_DML1_AIR_BM2.csv  
  inflating: /home/kim/data/bedmap_raw_data/bedmap2_raw_data/BEDMAP2 - Ice thickness, bed and surface elevation for Antarctica - standardised data points/AWI_1995_DML2_AIR_BM2.csv  
  inflating: /home/kim/data/bedmap_raw_data/bedmap2_raw_data/BEDMAP2 - Ice thickness, bed and surface elevation for Antarctica - standardised data points/AWI_1996_DML3_AIR_BM2.csv  
  inflating: /home/kim/data/bedmap_raw_data/bedmap2_raw_data/BEDMAP2 - Ice thickness, bed and surface elevation for Antarctica - standardised data points/AWI_1997_DML4_AIR_BM2.csv  
  inflating: /home/kim/data/bedmap_raw_data/bedmap2_raw_data/BEDMAP2 - Ice thickness, bed and surface elevation for Antarctica - standardised data points/AWI_1998_DML5_AIR_BM2.csv  
  inflating: /home/k

## Download Bedmap3

Bedmap3 is the largest collection of data and contains many csv files.

In [15]:
!wget -P "{path_to_bedmap3_data_folder}/bedmap3.zip" "https://ramadda.data.bas.ac.uk/repository/entry/show?entryid=91523ff9-d621-46b3-87f7-ffb6efcd1847&output=zip.tree"

--2025-05-15 11:35:25--  https://ramadda.data.bas.ac.uk/repository/entry/show?entryid=91523ff9-d621-46b3-87f7-ffb6efcd1847&output=zip.tree
Resolving ramadda.data.bas.ac.uk (ramadda.data.bas.ac.uk)... 194.66.5.50
Connecting to ramadda.data.bas.ac.uk (ramadda.data.bas.ac.uk)|194.66.5.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [multipart/x-zip]
Saving to: '/home/kim/data/bedmap_raw_data/bedmap3_raw_data/bedmap3.zip/show?entryid=91523ff9-d621-46b3-87f7-ffb6efcd1847&output=zip.tree’

show?entryid=91523f     [  <=>               ]   1.25G  5.03MB/s    in 3m 45s  

2025-05-15 11:39:14 (5.70 MB/s) - '/home/kim/data/bedmap_raw_data/bedmap3_raw_data/bedmap3.zip/show?entryid=91523ff9-d621-46b3-87f7-ffb6efcd1847&output=zip.tree’ saved [1343033701]



In [22]:
# unzip into the directory it is in
!unzip "{path_to_bedmap3_data_folder}/bedmap3.zip"

unzip:  cannot find or open /home/kim/data/bedmap_raw_data/bedmap3_raw_data/bedmap3.zip, /home/kim/data/bedmap_raw_data/bedmap3_raw_data/bedmap3.zip.zip or /home/kim/data/bedmap_raw_data/bedmap3_raw_data/bedmap3.zip.ZIP.


In [23]:
!pwd

/home/kim/ice_thickness


# Combine all data sets